[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# Covid-19 Dashboard

In [12]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [13]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [14]:
!pip install uk-covid19
from uk_covid19 import Cov19API
import json

In [18]:
filters = [
    'areaType=overview' # note each metric-value pair is inside one string
]
structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "cumDeaths28DaysByDeathDateRate"    
}
api = Cov19API(filters=filters, structure=structure)
timeseries=api.get_json()
print(timeseries)

{'data': [{'date': '2022-09-11', 'cases': None, 'hospital': 543, 'deaths': None}, {'date': '2022-09-10', 'cases': None, 'hospital': 451, 'deaths': None}, {'date': '2022-09-09', 'cases': None, 'hospital': 490, 'deaths': None}, {'date': '2022-09-08', 'cases': None, 'hospital': 588, 'deaths': None}, {'date': '2022-09-07', 'cases': None, 'hospital': 608, 'deaths': None}, {'date': '2022-09-06', 'cases': None, 'hospital': 635, 'deaths': None}, {'date': '2022-09-05', 'cases': None, 'hospital': 619, 'deaths': None}, {'date': '2022-09-04', 'cases': None, 'hospital': 534, 'deaths': None}, {'date': '2022-09-03', 'cases': None, 'hospital': 499, 'deaths': None}, {'date': '2022-09-02', 'cases': None, 'hospital': 548, 'deaths': None}, {'date': '2022-09-01', 'cases': None, 'hospital': 616, 'deaths': None}, {'date': '2022-08-31', 'cases': None, 'hospital': 648, 'deaths': None}, {'date': '2022-08-30', 'cases': None, 'hospital': 674, 'deaths': None}, {'date': '2022-08-29', 'cases': None, 'hospital': 676,

In [19]:
filters = [
    'areaType=nation',
    'areaName=England'
]


# values here are the names of the PHE metrics
structure = {
    "males": "maleCases",
    "females": "femaleCases"
}
api = Cov19API(filters=filters, structure=structure)
agedistribution=api.get_json()

In [20]:
with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)
with open("agedistribution.json", "wt") as OUTF:
    json.dump(agedistribution, OUTF)

In [21]:
with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)

In [22]:
datalist=data['data']
datalist
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()
dates

['2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03',
 '2020-02-04',
 '2020-02-05',
 '2020-02-06',
 '2020-02-07',
 '2020-02-08',
 '2020-02-09',
 '2020-02-10',
 '2020-02-11',
 '2020-02-12',
 '2020-02-13',
 '2020-02-14',
 '2020-02-15',
 '2020-02-16',
 '2020-02-17',
 '2020-02-18',
 '2020-02-19',
 '2020-02-20',
 '2020-02-21',
 '2020-02-22',
 '2020-02-23',
 '2020-02-24',
 '2020-02-25',
 '2020-02-26',
 '2020-02-27',
 '2020-02-28',
 '2020-02-29',
 '2020-03-01',
 '2020-03-02',
 '2020-03-03',
 '2020-03-04',
 '2020-03-05',
 '2020-03-06',
 '2020-03-07',
 '2020-03-08',
 '2020-03-09',
 '2020-03-10',
 '2020-03-11',
 '2020-03-12',
 '2020-03-13',
 '2020-03-14',
 '2020-03-15',
 '2020-03-16',
 '2020-03-17',
 '2020-03-18',
 '2020-03-19',
 '2020-03-20',
 '2020-03-21',
 '2020-03-22',
 '2020-03-23',
 '2020-03-24',
 '2020-03-25',
 '2020-03-26',
 '2020-03-27',
 '2020-03-28',
 '2020-03-29',
 '2020-03-30',
 '2020-03-31',
 '2020-04-01',
 '2020-04-02',
 '2020-04-03',
 '2020-04-04',
 '2020-04-05',
 '2020-04-

In [23]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [24]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
print (startdate, ' to ', enddate)

2020-01-31 00:00:00  to  2022-09-11 00:00:00


In [25]:
index=pd.date_range(startdate, enddate, freq='D')
timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])

In [26]:
for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    date=parse_date(entry['date'])
    for column in ['cases', 'hospital', 'deaths']:
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
        if pd.isna(timeseriesdf.loc[date, column]): 
            # replace None with 0 in our data 
            value= float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            timeseriesdf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
timeseriesdf.fillna(0.0, inplace=True)
            
timeseriesdf

,cases,hospital,deaths
2020-01-31,2.0,0.0,0.0
2020-02-01,0.0,0.0,0.0
2020-02-02,0.0,0.0,0.0
2020-02-03,0.0,0.0,0.0
2020-02-04,0.0,0.0,0.0
...,...,...,...
2022-09-07,0.0,608.0,0.0
2022-09-08,0.0,588.0,0.0
2022-09-09,0.0,490.0,0.0
2022-09-10,0.0,451.0,0.0


In [27]:
with open("agedistribution.json", "rt") as INFILE:
    data=json.load(INFILE)

In [28]:
datadic=data['data'][0] # data['data'] is a list
datadic

{'males': [{'age': '75_to_79', 'rate': 23001.8, 'value': 214854},
  {'age': '90+', 'rate': 39915.5, 'value': 67676},
  {'age': '50_to_54', 'rate': 33335.4, 'value': 637146},
  {'age': '65_to_69', 'rate': 23482.0, 'value': 316470},
  {'age': '40_to_44', 'rate': 41017.7, 'value': 709716},
  {'age': '85_to_89', 'rate': 29027.6, 'value': 103391},
  {'age': '15_to_19', 'rate': 40014.2, 'value': 640809},
  {'age': '20_to_24', 'rate': 38438.6, 'value': 688705},
  {'age': '60_to_64', 'rate': 28101.7, 'value': 440772},
  {'age': '35_to_39', 'rate': 39920.1, 'value': 739707},
  {'age': '80_to_84', 'rate': 22647.3, 'value': 144947},
  {'age': '70_to_74', 'rate': 20596.3, 'value': 276799},
  {'age': '10_to_14', 'rate': 46484.1, 'value': 818992},
  {'age': '30_to_34', 'rate': 41054.7, 'value': 786778},
  {'age': '0_to_4', 'rate': 14187.9, 'value': 235845},
  {'age': '45_to_49', 'rate': 36130.8, 'value': 651513},
  {'age': '25_to_29', 'rate': 39814.7, 'value': 766201},
  {'age': '55_to_59', 'rate': 

In [29]:
males=datadic['males']
females=datadic['females']
males # females is similar 

[{'age': '75_to_79', 'rate': 23001.8, 'value': 214854},
 {'age': '90+', 'rate': 39915.5, 'value': 67676},
 {'age': '50_to_54', 'rate': 33335.4, 'value': 637146},
 {'age': '65_to_69', 'rate': 23482.0, 'value': 316470},
 {'age': '40_to_44', 'rate': 41017.7, 'value': 709716},
 {'age': '85_to_89', 'rate': 29027.6, 'value': 103391},
 {'age': '15_to_19', 'rate': 40014.2, 'value': 640809},
 {'age': '20_to_24', 'rate': 38438.6, 'value': 688705},
 {'age': '60_to_64', 'rate': 28101.7, 'value': 440772},
 {'age': '35_to_39', 'rate': 39920.1, 'value': 739707},
 {'age': '80_to_84', 'rate': 22647.3, 'value': 144947},
 {'age': '70_to_74', 'rate': 20596.3, 'value': 276799},
 {'age': '10_to_14', 'rate': 46484.1, 'value': 818992},
 {'age': '30_to_34', 'rate': 41054.7, 'value': 786778},
 {'age': '0_to_4', 'rate': 14187.9, 'value': 235845},
 {'age': '45_to_49', 'rate': 36130.8, 'value': 651513},
 {'age': '25_to_29', 'rate': 39814.7, 'value': 766201},
 {'age': '55_to_59', 'rate': 30500.4, 'value': 565049},


In [30]:
ageranges=[x['age'] for x in males] # each entry of males is a dictionary
print(ageranges)

['75_to_79', '90+', '50_to_54', '65_to_69', '40_to_44', '85_to_89', '15_to_19', '20_to_24', '60_to_64', '35_to_39', '80_to_84', '70_to_74', '10_to_14', '30_to_34', '0_to_4', '45_to_49', '25_to_29', '55_to_59', '5_to_9']


In [31]:
def min_age(agerange):
    agerange=agerange.replace('+','') # remove the + from 90+
    start=agerange.split('_')[0]
    return int(start)

print(min_age('85_to_89'))
print(min_age('90+'))

85
90


In [32]:
ageranges.sort(key=min_age)
print (ageranges)

['0_to_4', '5_to_9', '10_to_14', '15_to_19', '20_to_24', '25_to_29', '30_to_34', '35_to_39', '40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84', '85_to_89', '90+']


In [33]:
age_df=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])

## Graphs and Analysis

In [34]:
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()